In [27]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd

# Prequisites

In [28]:
token = os.environ.get("INFLUXDB_TOKEN")
org = "th-koeln"
url = "http://localhost:8086"
bucket = "iot_telemetry_data"

# Transforming CSV for InfluxDB

In [ ]:
iot_tele_df = pd.read_csv("data/iot_telemetry_data.csv")
iot_tele_df = iot_tele_df.dropna()

# Convert the 'ts' column to datetime format
iot_tele_df['ts'] = pd.to_datetime(iot_tele_df['ts'], unit='s')

# Cut down to 20000 rows for testing
iot_tele_df = iot_tele_df.iloc[:20000]
iot_tele_df.tail()

,ts,device,co,humidity,light,lpg,motion,smoke,temp
19995,2020-07-12 09:30:20.689517975,00:0f:00:70:91:0a,0.002840,75.400002,False,0.005114,False,0.013275,19.5
19996,2020-07-12 09:30:21.783257246,b8:27:eb:bf:9d:51,0.004894,51.900000,False,0.007582,False,0.020214,21.5
19997,2020-07-12 09:30:25.044335127,1c:bf:ce:15:ec:4d,0.003866,74.699997,True,0.006392,False,0.016846,24.4
19998,2020-07-12 09:30:25.470076561,b8:27:eb:bf:9d:51,0.004888,51.900000,False,0.007575,False,0.020196,21.5
19999,2020-07-12 09:30:25.587088346,00:0f:00:70:91:0a,0.002840,75.400002,False,0.005114,False,0.013275,19.5


In [52]:
# Print date range
print("Date range: ", iot_tele_df['ts'].min(), " - ", iot_tele_df['ts'].max())

Date range:  2020-07-12 00:01:34.385974646  -  2020-07-12 09:30:25.587088346


In [ ]:
points = []

measurement_name = "iot_telemetry"

for index, row in iot_tele_df.iterrows():
    point = (
        Point(measurement_name)
        .tag("device", row["device"])
        .field("carbon_monoxide", float(row["co"]))
        .field("humidity", float(row["humidity"]))
        .field("light", bool(row["light"]))
        .field("liquid_petroleum_gas", float(row["lpg"]))
        .field("motion", bool(row["motion"]))
        .field("smoke", float(row["smoke"]))
        .field("temp", float(row["temp"]))
        .time(pd.to_datetime(row["ts"]), WritePrecision.NS) # pd.to_datetime already converted in cell ab2e96cd
    )
    points.append(point)

In [54]:
for i in range(min(3, len(points))): 
    print(points[i])

iot_telemetry,device=b8:27:eb:bf:9d:51 carbon_monoxide=0.0049559386483912,humidity=51,light=false,liquid_petroleum_gas=0.0076508222705571,motion=false,smoke=0.0204112701224129,temp=22.7 1594512094385974000
iot_telemetry,device=00:0f:00:70:91:0a carbon_monoxide=0.0028400886071015,humidity=76,light=false,liquid_petroleum_gas=0.005114383400977,motion=false,smoke=0.0132748367048515,temp=19.700000762939453 1594512094735567000
iot_telemetry,device=b8:27:eb:bf:9d:51 carbon_monoxide=0.0049760123404216,humidity=50.9,light=false,liquid_petroleum_gas=0.007673227406398,motion=false,smoke=0.0204751255761782,temp=22.6 1594512098073572000


In [ ]:
# Write data to InfluxDB with exception handling
try:
    write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
    write_api = write_client.write_api(write_options=SYNCHRONOUS)
    write_api.write(bucket=bucket, org=org, record=points)
    print("Data written to InfluxDB successfully!")
except Exception as e:
    print(f"Failed to write data to InfluxDB: {e}")
finally:
    if 'write_client' in locals() and write_client:
        write_client.close()

Data written to InfluxDB successfully!


# Reading the bucket

In [ ]:
# Read data from InfluxDB to verify and display as DataFrame
read_client = None  # Initialize to ensure it's defined for finally block
try:
    read_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
    query_api = read_client.query_api()

    # Modified query to pivot data for a structured DataFrame and read all data
    query = f'''from(bucket: "{bucket}")
      |> range(start: 0)
      |> filter(fn: (r) => r._measurement == "{measurement_name}")
      |> pivot(rowKey:["_time", "device"], columnKey: ["_field"], valueColumn: "_value")'''

    print('Querying data from InfluxDB and loading into DataFrame...')
    df_result = query_api.query_data_frame(query=query, org=org)

    final_df_to_print = None

    if isinstance(df_result, list):
        if not df_result:
            print('No data returned from query (empty list).')
        else:
            # Concatenate if df_result is a list of DataFrames
            concatenated_df = pd.concat(df_result, ignore_index=True)
            if not concatenated_df.empty:
                final_df_to_print = concatenated_df
            else:
                print('No data returned from query (empty concatenated DataFrame).')
    elif isinstance(df_result, pd.DataFrame):
        if not df_result.empty:
            final_df_to_print = df_result
        else:
            print('No data returned from query (empty DataFrame).')
    else:
        print('No data returned from query (data is None or unexpected type).')

    if final_df_to_print is not None:
        print("Query Result as DataFrame:")
        cols_to_drop = ['result', 'table']
        final_df_to_print = final_df_to_print.drop(columns=[col for col in cols_to_drop if col in final_df_to_print.columns], errors='ignore')
        print(final_df_to_print.head())

except Exception as e:
    print(f"Failed to read data from InfluxDB: {e}")
finally:
    if read_client:
        read_client.close()
        print('InfluxDB read client closed.')

Failed to read data from InfluxDB: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Encoding', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.11', 'X-Platform-Error-Code': 'invalid', 'Date': 'Wed, 07 May 2025 08:45:35 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: b'{"code":"invalid","message":"compilation failed: error @1:45-1:83: invalid expression @1:83-1:84: \'\\n\\nerror @1:53-1:83: invalid expression @1:52-1:53: \'\\n\\nerror @1:86-1:123: invalid expression @1:123-1:124: \'\\n\\nerror @1:93-1:123: invalid expression @1:92-1:93: \'"}'

InfluxDB read client closed.
